# Importation des bibliothèques

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy, os
import random as rd
import seaborn as sns
import pyodbc # connection base de données access
rd.seed(123)

# Méthodologie Questionnaire de Karasek

In [12]:
http_karasek='https://travail-emploi.gouv.fr/IMG/pdf/dares-karasek.pdf'
wiki_karasek='https://fr.wikipedia.org/wiki/Mod%C3%A8le_de_Karasek'
print(f'lien_questionnaire: {http_karasek}\nlien_wikipedia: {wiki_karasek}')

lien_questionnaire: https://travail-emploi.gouv.fr/IMG/pdf/dares-karasek.pdf
lien_wikipedia: https://fr.wikipedia.org/wiki/Mod%C3%A8le_de_Karasek


# Vérification de l'existence des drivers pour Microsoft Access 

In [13]:
''' Docstring
Il s'agit ici de vérifier l'existence des pilotes
*.mdb, *.accdb permettant à python de communiquer avec
la base de données access
'''
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

['Microsoft Access Driver (*.mdb, *.accdb)']

# Connection à la base de données

In [14]:
''' Docstring
Ici, vous pouvez changer le chemin d'accès <path> en
fonction de l'emplacement de votre base de données access.
Pour les paramètres de connections:
                        <UID>: identifiant
                        <PWD>: mot de passe
Ne rien saisir s'il n'y a pas de mot de passe et d'identifiant définis
et exécuter le code python tel qu'il est :) 
'''
# Chemin d'accès aux données
path=r"C:\Users\pc\Downloads\ENQUETE PREVALENCE STRESS\FatouAtegi\FatouAtegi.mdb"

# Saisie des Paramètres de connexion
connect_paras=(r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
               fr'DBQ={path};'
               r'UID=;' # saisir un ID S'il existe un identifiant d'accès à la base de données
               r'PWD=;' # saisir le mot de passe d'accès à la base de données
              )

#Connexion aux données
connect=pyodbc.connect(connect_paras)
curseur=connect.cursor()
Tables=curseur.tables(tableType='TABLE').fetchall()

# Accès aux tables de données "PREVALENCESTRESS"
Datas={}
for table in Tables:
    name=table.table_name.strip() # Pour extraire correctement le nom de la table
    if name.startswith('PREVALENCESTRESS'):
        query=f'SELECT * FROM [{name}]'
        Datas[name]=pd.read_sql(query,connect)
del Datas['PREVALENCESTRESS'] # Supprimer cette table d'information 

C:\Users\pc\AppData\Local\Temp\ipykernel_8488\2471132875.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Datas[name]=pd.read_sql(query,connect)


# Analyse Exploratoire des Données (EDA)

## Description et Fusion des tables de données "PREVALENCESTRESS"

### Tables de données importées

In [15]:
for name, data in Datas.items():
    print(name) # Noms des tables
    globals()[name]=data # extraction de chaque table

PREVALENCESTRESS1
PREVALENCESTRESS2
PREVALENCESTRESS3
PREVALENCESTRESS4


### Fonction pour décrire le DataFrame

In [16]:
def Des(df):
    # Vérification si l'objet df est bien un DataFrame
    if isinstance(df, pd.DataFrame):
        print(f"Yes :) data is a pandas DataFrame")
    else:
        print(f"{type(df).__name__} n'est pas un DataFrame pandas")
    
    # Création du DataFrame 'Information' 
    Information = pd.DataFrame({
        'Variables': df.columns,
        'Dtype': df.dtypes,
        'NaN_values': df.isna().sum(),
        'Unique_values': df.nunique()
    }).reset_index(drop=True)
    
    # Affiche toutes les lignes et colonnes temporairement
    with pd.option_context('display.max_columns', None, 'display.max_rows', None):
        print(Information)


### Fusion des tables de données

In [17]:
# Fusion des tables avec 'left_join' à partir de PREVALENCESTRESS1
liste=[PREVALENCESTRESS1, PREVALENCESTRESS2, PREVALENCESTRESS3, PREVALENCESTRESS4]

Donnees=liste[0].sort_values(by='GlobalRecordId')

for table in liste[1:]:
    table.sort_values(by='GlobalRecordId')
    Donnees=pd.merge(Donnees, table, on='GlobalRecordId', how='left') 

In [18]:
#Vérification de la réussite de la fusion des tables
size=0
for name, table in Datas.items():
    print(f'{name}: {table.shape[1]} variables')
    size+=table.shape[1]

#Nombre de variables attendus après la fusion
N=size-3 # on retire 3 car la variable de fusion 'GlobalRecordId' apparait une fois dans la base finale
print(N)

if Donnees.shape[1]==N:
    print('Fusion réussie :)')
else:
    raise ValueError("Le nombre total de variables attendus n'est pas conforme :(")

PREVALENCESTRESS1: 19 variables
PREVALENCESTRESS2: 16 variables
PREVALENCESTRESS3: 15 variables
PREVALENCESTRESS4: 17 variables
64
Fusion réussie :)


### Exporter les données vers un emplacement ou répertoire de travail au format 'Excel'

In [19]:
# Mon répertoire
Répertoire = r'C:\Users\pc\Downloads\ENQUETE PREVALENCE STRESS\FatouAtegi'
nom_fichier = 'Données brutes.xlsx' # !! Avec son extension .xlsx
chemin_complet = os.path.join(Répertoire, nom_fichier)

# Exporter le DataFrame au format Excel vers le répertoire
import openpyxl
Donnees.to_excel(chemin_complet, index=False, sheet_name='Tableau_1', engine='openpyxl')

## Analyse de la nouvelle base 'Donnees'

In [20]:
# information sur les variables
Des(Donnees)

Yes :) data is a pandas DataFrame
                                            Variables    Dtype  NaN_values  \
0                                      GlobalRecordId   object           0   
1                                                sexe   object           0   
2                                        Arretmaladie   object           0   
3                                                 age  float64           0   
4                                      HaurairesTache   object           0   
5                               SituationMatrimoniale   object           0   
6                                   ArreteDtravailler   object           0   
7                                           TitrePost   object           0   
8                                         HyginedeVie   object          12   
9                                         statusalari   object           0   
10                                        Typecontrat   object           0   
11                            

### Recodage des Variables de Réponses relatives au STRESS

In [21]:
# Fonction pour récoder
def Recode(X, inverse=True):
    '''--Docstring--
    Cette fonction a pour but d'uniformiser les labels au niveau du questionnaire de Karasek.
    Les labels présentent des caractères différents d'une question à une autre en raison des accents
    et autres caractères.
    l'argument inverse='True' affiche les variables catégorielles ordinales
    en utilisant des nombres (1-2-3-4) conformément à la methode de Karasek
    si inverse='False', alors les labels originels sont affichés
    [1-'fortement en désaccord ', 2-'en désaccord ', 3-"d'accord ", 4-"tout à fait d'accord "]
     l'argument par défaut est 'True' 
    '''
    import unidecode # Pour supprimer les accents et autres caractères
    
    # Répertoire de Travail pour exporter les données transformées en Excel.xlsx
    Répertoire = r'C:\Users\pc\Downloads\ENQUETE PREVALENCE STRESS\FatouAtegi'
    
    # Créer une copie profonde du DataFrame 
    X_copie=X.copy(deep=True)
    
    # Créer un dictionnaire à partir de mes labels dans cat_order
    dict={"'":"", "’":""}

    cat_order=['fortement en desaccord','en desaccord','daccord', 'tout a fait daccord']    
    
    labels={j:(i+1) for i,j in enumerate(cat_order)}
    
    #Vérifier que X est bien un DataFrame
    if isinstance(X_copie, pd.DataFrame):
        pass
    else:
        raise TypeError(f"{type(X_copie).__name__} n'est pas un DataFrame pandas")

    # Itération sur les colonnes'type objet' pour convertir en catégorie
    for var in X_copie.iloc[:,1:].columns:
        
        #Pour toutes les variables catégorielles nominales
        if (X_copie[var].dtype=='object') and not (var in X_copie.columns.tolist()[19:61]):
            list_a=[label for label in X_copie[var].unique() if label != None]
            X_copie[var]=X_copie[var].astype('category')
            X_copie[var]=pd.Categorical(X_copie[var],
                                        categories=list_a,
                                        ordered=False)
            
        # pour les variables Catégorielles ordinales
        if (X_copie[var].dtype=='object') and (var in X_copie.columns.tolist()[19:61]):
             # supression des accents et caractères et espaces dans les labels
            X_copie[var]=X_copie[var].apply(
                lambda x: unidecode.unidecode(x).strip().translate(str.maketrans(dict))
                          if x!=None else x)
            X_copie[var]=X_copie[var].astype('category')
            X_copie[var]=pd.Categorical(X_copie[var],
                                        categories=cat_order,
                                        ordered=True)
            # Appliquer les transformations pour inverse='True' 
            if inverse:
                X_copie[var] = X_copie[var].map(labels)
                 
    #retourne la copie transformée du DataFrame
    return X_copie

### Application de la fonction recode() à mes données

In [22]:
# Traitement des données via notre fonction <recode()>
data=Recode(Donnees,True)

### Traitement des valeurs manquantes 'NA'

In [23]:
# Imputation des des variables ayant des 'NA'
data_imputed=data.loc[:,'sexe':] # exclusion de la variable 'GlobalRecordId'

for var in data_imputed.columns:
    if data_imputed[var].isna().any():
        mode=data_imputed[var].mode()[0] # mode de la série
        data_imputed[var]=data_imputed[var].fillna(mode) # remplacer les 'NA' par le mode


### Création de la variable groupe d'âge

In [24]:
# Statistiques descriptives de la variable Âge
(data_imputed['age']).describe()

count    65.000000
mean     36.015385
std      11.857872
min      18.000000
25%      26.000000
50%      33.000000
75%      48.000000
max      62.000000
Name: age, dtype: float64

In [25]:
# Création de la variable groupe d'âge
data_imputed['group_age']=pd.cut(data_imputed['age'],
                        bins=[17,31,41,51,100],
                        labels=['18-30 ans','31-40 ans','41-50 ans','+50 ans'])

## Calcul des Scores de Karasek

### Score pour la demande psychologique

In [26]:
"""--Docstring--
L’axe « Demande psychologique » regroupe:
>>Quantité - rapidité
Q10 - Mon travail me demande de travailler très vite
Q12 - On me demande d’effectuer une quantité de travail excessive
Q13 - Je dispose du temps pour exécuter correctement mon travail

>>Complexité - intensité
Q14 - Je reçois des ordres contradictoires de la part d’autres personnes
Q11 - Mon travail me demande de travailler intensément
Q15 - Mon travail demande de longues périodes de concentration intense

>>Morcellement, prévisibilité
Q16 - Mes tâches sont souvent interrompues avant d’être achevées, nécessitant de les reprendre plus tard
Q17 - Mon travail est très bousculé
Q18 - Attendre le travail de collègues ou d’autres départements ralentit souvent mon propre travail

>>Le score de demande psychologique est donné par la formule :
            Q10+Q11+Q12+(5-Q13)+Q14+Q15+Q16+Q17+Q18
"""

'--Docstring--\nL’axe « Demande psychologique » regroupe:\n>>Quantité - rapidité\nQ10 - Mon travail me demande de travailler très vite\nQ12 - On me demande d’effectuer une quantité de travail excessive\nQ13 - Je dispose du temps pour exécuter correctement mon travail\n\n>>Complexité - intensité\nQ14 - Je reçois des ordres contradictoires de la part d’autres personnes\nQ11 - Mon travail me demande de travailler intensément\nQ15 - Mon travail demande de longues périodes de concentration intense\n\n>>Morcellement, prévisibilité\nQ16 - Mes tâches sont souvent interrompues avant d’être achevées, nécessitant de les reprendre plus tard\nQ17 - Mon travail est très bousculé\nQ18 - Attendre le travail de collègues ou d’autres départements ralentit souvent mon propre travail\n\n>>Le score de demande psychologique est donné par la formule :\n            Q10+Q11+Q12+(5-Q13)+Q14+Q15+Q16+Q17+Q18\n'

In [27]:
#Création de la liste des variables concernées 
Psy=data.loc[:,'N10Montravailexigedallertrsvite':'N18JesuiRalentiTravailParceqJedoisAttendreAutres']
Psy=Psy.columns.tolist()
Psy.remove('N13JaiSuffisammentDtempsTravail')
Psy

['N10Montravailexigedallertrsvite',
 'N11TravaillerTrsfortMentalement',
 'N12OnNedemandepasDfaireQuantitExcessiveDtravail',
 'N14PasDdemandesContradictoires',
 'N15ConcentrerIntensmentPendLonguesPeriodes',
 'N16MatachtInterrompueAvanTerminJdoisRevenirPlusTard',
 'N17MonTravailTresSouvenMouvement',
 'N18JesuiRalentiTravailParceqJedoisAttendreAutres']

In [28]:
# Calcul du Score
data_imputed['Score_demande_psychologique']=(data_imputed.loc[:,Psy].astype('float').sum(axis=1) +
                            (5 - data_imputed['N13JaiSuffisammentDtempsTravail'].astype('float')))

### Score pour la latitude décisionnelle 

In [29]:
"""--docstring--
L’axe « latitude décisionnelle »» regroupe:
>>Latitude ou marges de manœuvre
Q4 - Mon travail me permet de prendre souvent des décisions moi-même
Q6 - Dans ma tâche, j’ai très peu de libertés pour décider comment je fais mon travail
Q8 - J’ai la possibilité d’influencer le déroulement de mon travail

>>Utilisation actuelle des compétences
Q2 - Dans mon travail, j’effectue des tâches répétitives
Q5 - Mon travail demande un haut niveau de compétence
Q7 - Dans mon travail, j’ai des activités variées

>>Développement des compétences
Q1 - Dans mon travail, je dois apprendre des choses nouvelles
Q3 - Mon travail me demande d’être créatif
Q9 - J’ai l’occasion de développer mes compétences professionnelles

>>Le score de latitude décisionnelle est donné par la formule :
                4*Q4+4*(5-Q6)+4*(Q8)+2*(5-Q2)+2*(Q5)+2*(Q7)+ 2*(Q1)+2*(Q3)+2*(Q9)
"""

'--docstring--\nL’axe « latitude décisionnelle »» regroupe:\n>>Latitude ou marges de manœuvre\nQ4 - Mon travail me permet de prendre souvent des décisions moi-même\nQ6 - Dans ma tâche, j’ai très peu de libertés pour décider comment je fais mon travail\nQ8 - J’ai la possibilité d’influencer le déroulement de mon travail\n\n>>Utilisation actuelle des compétences\nQ2 - Dans mon travail, j’effectue des tâches répétitives\nQ5 - Mon travail demande un haut niveau de compétence\nQ7 - Dans mon travail, j’ai des activités variées\n\n>>Développement des compétences\nQ1 - Dans mon travail, je dois apprendre des choses nouvelles\nQ3 - Mon travail me demande d’être créatif\nQ9 - J’ai l’occasion de développer mes compétences professionnelles\n\n>>Le score de latitude décisionnelle est donné par la formule :\n                4*Q4+4*(5-Q6)+4*(Q8)+2*(5-Q2)+2*(Q5)+2*(Q7)+ 2*(Q1)+2*(Q3)+2*(Q9)\n'

In [30]:
# Liste des variables concernées
latitude=["N6PrendrDcisionsDfaconAutonom","N5LibertDdciderComTravail","N8PeutDinfluencesTravail"]
utilisation=["N4RefairetoujoursMemChoses","NiveauEleveDqualifications","N7fairePlusieursChosesDifftes"]
competence=["ApprendreChosesNouvelles","PreuveDcreativit","N9DveloppHabiletsPerso"] 

In [31]:
# Calul du Score
data_imputed['Score_latitude_decisionnelle']=(data_imputed[latitude[0]].astype('float')*4 +
                                (5 - data_imputed[latitude[1]].astype('float'))*4 +
                                data_imputed[latitude[2]].astype('float')*4 +
                                (5 - data_imputed[utilisation[0]].astype('float'))*4 +
                                data_imputed[utilisation[1]].astype('float')*2 +
                                data_imputed[utilisation[2]].astype('float')*2 +
                                (data_imputed.loc[:,competence].astype('float').sum(axis=1))*2)

### Score pour le soutien social

In [32]:
"""--Docstring--
L’axe «soutien social» distingue le soutien professionnel ou émotionnel,
en provenance des supérieurs ou des collègues :

>>Le soutien professionnel- par les supérieurs :
Q22 - Mon supérieur réussit facilement à faire collaborer ses subordonnés
Q21 - Mon supérieur m’aide à mener ma tâche à bien- par les collègues :
Q23 - Les collègues avec qui je travaille sont des gens professionnellement compétents
Q26 - Les collègues avec qui je travaille m’aident à mener les tâches à bien

>>Le soutien émotionnel :- par les supérieurs
Q20 - Mon supérieur prête attention à ce que je dis
Q19 - Mon supérieur se sent concerné par le bien-être de ses subordonnés- par les collègues
Q25 - Les collègues avec qui je travaille sont amicaux
Q24 - Les collègues avec qui je travaille me manifestent de l’intérêt

>>Le score de soutien social est donné par la formule:
                            Q19+Q20+Q21+Q22+Q23+Q24+Q25+Q26
"""

'--Docstring--\nL’axe «soutien social» distingue le soutien professionnel ou émotionnel,\nen provenance des supérieurs ou des collègues :\n\n>>Le soutien professionnel- par les supérieurs :\nQ22 - Mon supérieur réussit facilement à faire collaborer ses subordonnés\nQ21 - Mon supérieur m’aide à mener ma tâche à bien- par les collègues :\nQ23 - Les collègues avec qui je travaille sont des gens professionnellement compétents\nQ26 - Les collègues avec qui je travaille m’aident à mener les tâches à bien\n\n>>Le soutien émotionnel :- par les supérieurs\nQ20 - Mon supérieur prête attention à ce que je dis\nQ19 - Mon supérieur se sent concerné par le bien-être de ses subordonnés- par les collègues\nQ25 - Les collègues avec qui je travaille sont amicaux\nQ24 - Les collègues avec qui je travaille me manifestent de l’intérêt\n\n>>Le score de soutien social est donné par la formule:\n                            Q19+Q20+Q21+Q22+Q23+Q24+Q25+Q26\n'

In [33]:
#Liste des variables concernées
soutien_prof=['N22MchefFaciliRealisationDuTravail','N23MchefRuessiFaireTavaillerGensEnsemble',
             'N24LesGensAvecQJTravailleSontQualifies','N29LesgenAcqjetravaillefacilitentlaralisationdutravail']

soutien_emo=['N20MchefprteAttentionCquejdis','N19MchefSeSoucieBientreTravailleurs',
            'N27Lesgensavecquijetravaillesontamicaux','N25LesGensAvecQjTravailleSintressentMoi']

liste_result = soutien_prof + soutien_emo

In [34]:
# Calcul du Score
data_imputed['Score_soutien_social'] = data_imputed.loc[:,liste_result].astype('float').sum(axis=1)

## Calcul des 4 Etats psychologiques de Karasek

In [35]:
"""--Docstring-- Les 4 Etats psychologiques de Karasek

Le travail actif : forte autonomie et forte exigence. 
Le salarié, même si une certaine pression est présente, 
dispose de suffisamment d’autonomie pour l’absorber et organiser son travail.

Le travail passif : faible autonomie et faible exigence. 
Le salarié n’a que peu de marge de manœuvre en termes d’organisation de son travail
mais ne subit pas de pression particulière.

Le travail détendu : forte autonomie et faible exigence. 
Dans ce cas, la pression est faible et le salarié a beaucoup de latitude 
pour s’organiser et faire le travail qui lui est demandé.

Le travail tendu : faible autonomie et forte exigence. La pire des situations. 
Le salarié subit une importante pression psychologique tout en n’ayant quasiment 
aucune marge de manœuvre pour s’adapter.

"""

'--Docstring-- Les 4 Etats psychologiques de Karasek\n\nLe travail actif : forte autonomie et forte exigence. \nLe salarié, même si une certaine pression est présente, \ndispose de suffisamment d’autonomie pour l’absorber et organiser son travail.\n\nLe travail passif : faible autonomie et faible exigence. \nLe salarié n’a que peu de marge de manœuvre en termes d’organisation de son travail\nmais ne subit pas de pression particulière.\n\nLe travail détendu : forte autonomie et faible exigence. \nDans ce cas, la pression est faible et le salarié a beaucoup de latitude \npour s’organiser et faire le travail qui lui est demandé.\n\nLe travail tendu : faible autonomie et forte exigence. La pire des situations. \nLe salarié subit une importante pression psychologique tout en n’ayant quasiment \naucune marge de manœuvre pour s’adapter.\n\n'

In [36]:
# Fonction de calcul des 4 états

def karasek_etat(row):
    # valeur de référence 'médiane' pour la demande psychologique
    demande_ref=data_imputed['Score_demande_psychologique'].median()
    
    # valeur de référence 'médiane' pour la latitude décisionnelle  
    latitude_ref=data_imputed['Score_latitude_decisionnelle'].median()
    
    if row['Score_demande_psychologique'] > demande_ref and row['Score_latitude_decisionnelle'] > latitude_ref:
        return 'Etat_Actif'
    elif row['Score_demande_psychologique'] > demande_ref and row['Score_latitude_decisionnelle'] <= latitude_ref:
        return 'Etat_Stress'
    elif row['Score_demande_psychologique'] <= demande_ref and row['Score_latitude_decisionnelle'] > latitude_ref:
        return 'Etat_Détendu'
    else:
        return 'Etat_Passif'

data_imputed['Etat_Karasek'] = data_imputed.apply(karasek_etat, axis=1)
data_imputed['Etat_Karasek'] = data_imputed['Etat_Karasek'].astype('category')

## Calcul des Scores Efforts/Récompenses suivant le modèle de SIGRIEST

In [37]:
# Documentation sur le modèle de SIEGRIST
lien_1="http://gemto.free.fr/qessiegrist.pdf"
print(lien_1)

http://gemto.free.fr/qessiegrist.pdf


In [38]:
# Définir les colonnes d'effort et de récompense
Efforts_items = ['N1JesuisConstPrisparLTempsCausChargeTravail',
                 'N10Montravailexigedallertrsvite',
                 'N11TravaillerTrsfortMentalement',
                 'N12MonTravaiEexigeDeffortsPhysiques',
                 'N13CesDernieresAnnesMTravailEstDevenuPlusExigeant']

Recompenses_items = ['N4JeRecoisRespectQjeMriteDmesSuprieurs',
                     'N7JeRecoisRespectQjeMeriteDmesCollgues',
                     'N10VuMesEffortsMesPerspectivDPromotionSatisfaisant',
                     'N11VuMesEffortMsalaireEstSatisfaisant',
                     'N5MaScuritDemploiEstLenace']

# Calculer la somme des scores d'effort et de récompense pour chaque ligne
data_imputed['Score_Efforts'] = data_imputed[Efforts_items].astype('float').sum(axis=1)
data_imputed['Score_Récompenses'] = data_imputed[Recompenses_items].astype('float').sum(axis=1)

# Facteur de correction basé sur le nombre d'items
correction = len(Recompenses_items) / len(Efforts_items)

# Calculer du rapport Equilibre_Efforts_Recompenses
data_imputed['Equilibre_Efforts_Récompenses'] = (data_imputed['Score_Efforts'] / data_imputed['Score_Récompenses']) * (correction)


## Renommer correctement certaines colonnes

In [39]:
data_imputed.rename(columns={'HaurairesTache':'Horaires_tâches',
                            'HyginedeVie':'Hygiène_vie',
                             'Arretmaladie':"Arrêt_maladie",
                            'ArreteDtravailler':"Arrêt_travail",
                            'TitrePost':'Titre_Post',
                            'statusalari':'Statut_salarié',
                            'conflitHierarchie':'Conflit_Hiérarchie',
                            'MaladiesProfl':'Maladies_Profil',
                            'Conflitcollgues':'Conflit_collègues',
                            'PasDproblmeSante':'Pas_de_problème_Santé',
                            'Conflitsubordonns':'Conflit_subordonnées',
                            'PasDconflit':'Pas_de_conflit'}, inplace=True)

# Remplacer salariré par salarié 
data_imputed['Statut_salarié'] = np.where(data_imputed['Statut_salarié'] == 'Salariré',
                                          'Salarié', data_imputed['Statut_salarié'])
data_imputed['Statut_salarié']=data_imputed['Statut_salarié'].astype('category')

## Exportation des données finales "avec les scores calculés"

In [40]:
data_imputed.to_excel(os.path.join(Répertoire,'Data_cleaned.xlsx'),
                      index=False, sheet_name='Tableau', engine='openpyxl')

#  Visualisation des données & Analyses Statistiques 

## Visualisation des Données

### Création de listes de variables pour les calculs

In [41]:
# Création de listes de variables pour les répartitions statistiques 
cat_list=data_imputed.loc[:,'sexe':'Pas_de_conflit'].columns.tolist()
cat_list.remove('age')
cat_list.insert(1,'group_age')

#Liste pour les scores de Karasek
var_list=data_imputed.loc[:,'Score_demande_psychologique':'Score_soutien_social'].columns.tolist()


### Répartition des caractéristiques sociodémographiques

In [42]:
from docx import Document
from docx.shared import Inches
from docx.enum.section import WD_ORIENT

# Créer un document Word
doc = Document()

# Mettre le document en format paysage
section = doc.sections[-1]
section.orientation = WD_ORIENT.LANDSCAPE
section.page_width, section.page_height = section.page_height, section.page_width

# Initialisation d'un dictionnaire pour stocker les tables de contingences
Pivot = {}

# Calcul des tables de contingence pour chaque variable catégorielle
for var in cat_list:
    if var not in ['Titre_Post', 'Hygiène_vie', 'Maladies_Profil']:
        pivot_table = data_imputed[var].astype('category').value_counts(normalize=True) * 100
        Pivot[var] = pivot_table

# Création et exportation des graphiques en barres
for var, value in Pivot.items():
    sns.set(style="whitegrid")
    palette = sns.color_palette("husl", len(value))
    
    table = pd.DataFrame({'Catégories': value.index, 'Pourcentage': value.values})
    
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x='Catégories', y='Pourcentage', hue='Catégories',
                     data=table, dodge=False)
    ax.set_xlabel('Catégories', fontsize=12, weight='bold')
    ax.set_ylabel('Pourcentage (%)', fontsize=12, weight='bold')
    ax.set_title(f'Répartition par {var}', fontsize=14, weight='bold')
    
    for p in ax.patches:
        percentage = p.get_height()
        text_offset = 10 if percentage > 5 else percentage / 2
        ax.annotate(f'{percentage:.1f}%',
                    (p.get_x() + p.get_width() / 2., p.get_height() - text_offset),
                    ha='center', va='center', 
                    fontsize=12, color='white', fontweight='bold')
    
    sns.despine(left=True, bottom=True)
    
    image_path = f'{var}_graphique.png'
    plt.savefig(image_path)
    plt.close()
    
    doc.add_picture(image_path, width=Inches(9))

# Création des camemberts pour les variables spécifiques
for var in ['Titre_Post', 'Hygiène_vie']:
    pivot_table = data_imputed[var].astype('category').value_counts(normalize=True) * 100
    
    plt.figure(figsize=(8, 8))
    plt.pie(pivot_table, labels=pivot_table.index, autopct='%1.1f%%', startangle=140,
            colors=sns.color_palette("husl", len(pivot_table)))
    plt.title(f'Répartition par {var}', fontsize=14, weight='bold')
    image_path = f'{var}_camembert.png'
    plt.savefig(image_path)
    plt.close()
    
    doc.add_picture(image_path, width=Inches(6))

# Sauvegarder le document Word
file_name = 'Caractéristiques Sociodémographiques graphiques.docx'
doc.save(os.path.join(Répertoire, file_name))


ModuleNotFoundError: No module named 'docx'

### Visualisation des Scores de Karasek en fonction des caractéristiques sociodémographiques

In [ ]:
# Créer un document Word
doc_1 = Document()

# Mettre le document en format paysage
section = doc_1.sections[-1]
section.orientation = WD_ORIENT.LANDSCAPE
section.page_width, section.page_height = section.page_height, section.page_width
# Liste des variables à tracer
variables = cat_list[0:10]

# Tracer les données pour chaque variable
for var in variables:
    if var in data_imputed.columns:
        plt.figure(figsize=(10, 6))
        
        # Créer un scatter plot pour la variable en cours
        sns.scatterplot(data=data_imputed, x='Score_latitude_decisionnelle', y='Score_demande_psychologique',
                        hue=var, style=var, s=100)
        
        # Ajouter des lignes pour séparer les quadrants
        plt.axvline(x=data_imputed['Score_latitude_decisionnelle'].median(), color='gray', linestyle='--')
        plt.axhline(y=data_imputed['Score_demande_psychologique'].median(), color='gray', linestyle='--')

        # Ajouter des annotations pour chaque quadrant
        plt.text(data_imputed['Score_latitude_decisionnelle'].min(), data_imputed['Score_demande_psychologique'].max(),
                 'TENDUS', color='purple', fontsize=14)
        plt.text(data_imputed['Score_latitude_decisionnelle'].max(), data_imputed['Score_demande_psychologique'].max(),
                 'ACTIFS', color='teal', fontsize=14, ha='right')
        plt.text(data_imputed['Score_latitude_decisionnelle'].min(), data_imputed['Score_demande_psychologique'].min(),
                 'PASSIFS', color='teal', fontsize=14)
        plt.text(data_imputed['Score_latitude_decisionnelle'].max(), data_imputed['Score_demande_psychologique'].min(),
                 'DETENDUS', color='purple', fontsize=14, ha='right')

        # Labels et titre
        plt.title(f'Analyse par {var}')
        plt.xlabel('Latitude décisionnelle')
        plt.ylabel('Demande psychologique')

        # Ajouter une source au graphique
        plt.figtext(0.01, 0.01, "Source: Données d'enquêtes extraites de EPI", horizontalalignment='left', fontsize=8, color='black')
        plt.legend(title=var, loc='upper right')
        
        # Sauvegarder le graphique en tant que fichier PNG
        image_path = f'Analyse par {var}.png'
        plt.savefig(image_path)
        plt.close()
    
        doc_1.add_picture(image_path, width=Inches(9))

# Sauvegarder le document Word
file_name = 'Scores de Karasek et caractéristiques socio.docx'
doc_1.save(os.path.join(Répertoire, file_name))


## Analyses et tables statistiques 

### Tableaux statistiques Scores de Karasek et caractéristiques socio 

#### Fonction pour calculer tableau statistique Général sur l'échantillon 

In [ ]:

def statistical_summary(data, variables_list):
    # Validation des données
    if not isinstance(data, pd.DataFrame):
        raise TypeError('data is not a pandas.DataFrame')

    summary_table = pd.DataFrame()

    # Calcul des statistiques pour chaque variable
    for var in variables_list:
        if not var in data.columns:
            raise ValueError(f'{var} not in data.columns')
        if not data[var].dtype in ['float','numeric']:
            raise TypeError(f'{var} is not a numeric/float dtype')

        summary = {
            'Min': data[var].min(),
            'Max': data[var].max(),
            'Mean': data[var].mean(),
            'Median': data[var].median(),
            'Variance': data[var].var(),
            'Std Dev': data[var].std(),
            'Total Observations': data[var].count()
        }

        summary_table[var] = pd.Series(summary)

    return summary_table.T  # Transposer pour avoir les variables en ligne


#### Fonction pour calculer Tableau Statistique Croisé avec variables catégorielles

In [ ]:

def Cross_summary(data,targets_list,features_list):
    
    #librairie pour transformer une liste imbriquée en liste simple
    from itertools import chain
    
    # Validation des éléments d'entrée
    if not isinstance(data, pd.DataFrame):
        raise TypeError('data is not a pandas.DataFrame')
    
    for var in targets_list: # Variables continues ou numeriques
        if not (var in data.columns and data[var].dtype in ['float','numeric']):
            raise ValueError(f'{var} not in data.columns or not in dtype.["float","numeric"]')
    
    for var in features_list: # Variables catégorielles
        if not (var in data.columns and data[var].dtype in ['category']):
            raise ValueError(f'{var} not in data.columns or not in dtype.["category"]')

    labels = [data[var].unique() for var in features_list]
    labels = list(chain.from_iterable(labels)) # transformer les listes imbriquées en une liste unique
    a, b = labels.index('31-40 ans'), labels.index('41-50 ans')
    labels[a], labels[b]=labels[b],labels[a] # intervertir les places de deux labels 

    #Création d'un dictionnaire de mapping entre les labels et leur variable 
    Group_index={}
    
    for var in features_list:
        Group_index.update({label:f'{var}' for label in data[var].unique()})

    for score in targets_list:
        # Fusionner les catégories dans une colonne avec 'melt'
        data_melted = data.melt(id_vars=score, value_vars=features_list,
                          var_name=f'Statistiques: {score}', value_name='Categories')
    
        #Nombres d'observations par catégories
        obs=data_melted.groupby('Categories').size()
    
        # Calculer les statistiques descriptives pour chaque catégorie
        table_1 = data_melted.groupby('Categories')[score].agg(['min', 'max', 'mean', 'median', 'var','std'])
        table_1['obs']=obs

        #Les distributions de percentiles 1% 25% 50% 75% 99%
        table_2 = data_melted.groupby('Categories')[score].quantile([0.01,0.25,0.50,0.75,0.99]).unstack()
        table_2['obs']=obs
    
        # Renommer les colonnes
        table_1.columns = ['Minimum', 'Maximum', 'Mean', 'Median', 'Variance',
                       'Standart-deviation',"Observations par Categories"]
        table_2.columns = ['Percentile 1%','Percentile 25%', 'Percentile 50%', 
                       'Percentile 75%','Percentile 99%',"Observations par categories"]
    
        #Trier les valeurs des catégories 
        table_1.index=pd.Categorical(table_1.index, categories=labels, ordered=True)
        table_1=table_1.sort_index()
        table_2.index=pd.Categorical(table_2.index, categories=labels, ordered=True)
        table_2=table_2.sort_index()

        # Créer une nouvelle colonne qui contient les groupes en utilisant le mapping
        table_1['Caractéristiques'] = table_1.index.map(Group_index)
        table_2['Caractéristiques'] = table_2.index.map(Group_index)

        # Maintenant, créer un multi-index en utilisant le groupe et les index existants
        table_1 = table_1.set_index(['Caractéristiques', table_1.index])
        table_2 = table_2.set_index(['Caractéristiques', table_2.index])

        #Enregistrer les tables dans les dictionnaires correspondants
        print(f"TABLEAU STATISTIQUE DE {score} PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES")
        display(table_1)
        print(f"\n")
        print(f"TABLEAU DES DECILES DE {score} PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES")
        display(table_2)

#### Application des deux fonctions sur mes plages de valeurs

In [ ]:
print(f"\n")
print("TABLEAU STATISTIQUE GENERAL SUR LES SCORES DE KARASEK DANS L'ECHANTILLON TOTAL")
display(statistical_summary(data_imputed, var_list))
print(f"\n")
Cross_summary(data_imputed,var_list,cat_list[0:10])



TABLEAU STATISTIQUE GENERAL SUR LES SCORES DE KARASEK DANS L'ECHANTILLON TOTAL


,Min,Max,Mean,Median,Variance,Std Dev,Total Observations
Score_demande_psychologique,15.0,31.0,25.538462,26.0,11.596154,3.405313,65.0
Score_latitude_decisionnelle,62.0,92.0,74.215385,74.0,48.015385,6.929313,65.0
Score_soutien_social,16.0,32.0,25.800000,26.0,12.693750,3.562829,65.0




TABLEAU STATISTIQUE DE Score_demande_psychologique PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Minimum  Maximum  \
Caractéristiques                                                           
sexe                  Homme                                23.0     31.0   
                      Femme                                15.0     31.0   
group_age             18-30 ans                            15.0     31.0   
                      31-40 ans                            22.0     31.0   
                      41-50 ans                            19.0     31.0   
                      +50 ans                              22.0     31.0   
Arrêt_maladie         Aucune fois                          19.0     31.0   
                      Une fois                             15.0     31.0   
                      Deux fois ou plus                    19.0     26.0   
Horaires_tâches       Le temps prescrit                    15.0     29.0   
                      Plus                                 19.0     31.0   
                      Moins                                20.0     28.0   
SituationMatrimoniale Marié                                15.0     31.0   
                      Célibataire                          20.0     31.0   
Arrêt_travail         Jamais                               21.0     31.0   
                      Rarement                             19.0     31.0   
                      À certain moments                    20.0     31.0   
                      N'importe quand                      15.0     28.0   
Titre_Post            D.E.S.                               23.0     31.0   
                      Infirmiers                           15.0     28.0   
                      Stagiaires                           20.0     29.0   
                      Médecin spécialiste                  28.0     28.0   
                      Médecin généraliste                  22.0     29.0   
                      Aides de bloc opératoire             27.0     31.0   
                      Sage-femme                           19.0     31.0   
                      Professeur Med                       23.0     30.0   
                      Internes ou Thésards                 22.0     23.0   
                      Chargé de nettoyage                  28.0     31.0   
                      Anesthésiste réanimateur             23.0     27.0   
Hygiène_vie           Pratique d'une activité sportive     15.0     31.0   
                      Consommation d'alcool                23.0     26.0   
                      Consommation de somnifères           24.0     28.0   
                      Consommation de tabac                23.0     23.0   
Statut_salarié        Bénévole                             20.0     31.0   
                      Salarié                              15.0     31.0   
Typecontrat           Aucun contrat                        20.0     31.0   
                      Stage                                21.0     29.0   
                      CDI                                  15.0     31.0   

                                                             Mean  Median  \
Caractéristiques                                                            
sexe                  Homme                             27.000000    27.0   
                      Femme                             24.682927    25.0   
group_age             18-30 ans                         24.892857    25.0   
                      31-40 ans                         26.875000    27.0   
                      41-50 ans                         25.000000    25.0   
                      +50 ans                           26.142857    25.0   
Arrêt_maladie         Aucune fois                       26.056604    26.0   
                      Une fois                          23.400000    23.5   
                      Deux fois ou plus                 22.500000    22.5   
Horaires_tâches       Le temps prescrit                 24.962963    25.0   
                      Plus                              26.200000    27.0   
              



TABLEAU DES DECILES DE Score_demande_psychologique PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


...

Percentile 1%  \
Caractéristiques                                                        
sexe                  Homme                                     62.46   
                      Femme                                     64.80   
group_age             18-30 ans                                 66.00   
                      31-40 ans                                 62.30   
                      41-50 ans                                 64.26   
                      +50 ans                                   66.12   
Arrêt_maladie         Aucune fois                               63.04   
                      Une fois                                  68.00   
                      Deux fois ou plus                         64.06   
Horaires_tâches       Le temps prescrit                         66.52   
                      Plus                                      62.68   
                      Moins                                     70.20   
SituationMatrimoniale Marié                                     62.92   
                      Célibataire                               66.00   
Arrêt_travail         Jamais                                    64.32   
                      Rarement                                  62.20   
                      À certain moments                         66.00   
                      N'importe quand                           68.12   
Titre_Post            D.E.S.                                    62.16   
                      Infirmiers                                66.20   
                      Stagiaires                                68.16   
                      Médecin spécialiste                       68.00   
                      Médecin généraliste                       70.12   
                      Aides de bloc opératoire                  74.18   
                      Sage-femme                                64.64   
                      Professeur Med                            70.20   
                      Internes ou Thésards                      68.02   
                      Chargé de nettoyage                       64.08   
                      Anesthésiste réanimateur                  66.36   
Hygiène_vie           Pratique d'une activité sportive          63.18   
                      Consommation d'alcool                     70.02   
                      Consommation de somnifères                68.08   
                      Consommation de tabac                     80.00   
Statut_salarié        Bénévole                                  64.66   
                      Salarié                                   62.60   
Typecontrat           Aucun contrat                             64.38   
                      Stage                                     68.00   
                      CDI                                       62.54   

                                                        Percentile 25%  \
Caractéristiques                                                         
sexe                  Homme                                       66.0   
                      Femme                                       70.0   
group_age             18-30 ans                                   69.5   
                      31-40 ans                                   67.5   
                      41-50 ans                                   70.0   
                      +50 ans                                     68.0   
Arrêt_maladie         Aucune fois                                 68.0   
                      Une fois                                    70.0   
                      Deux fois ou plus                           65.5   
Horaires_tâches       Le temps prescrit                           70.0   
                      Plus                                        67.0   
                      Moins                                       75.0   
SituationMatrimoniale Marié                                       69.0   
                      Cél

TABLEAU STATISTIQUE DE Score_soutien_social PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Minimum  Maximum  \
Caractéristiques                                                           
sexe                  Homme                                20.0     32.0   
                      Femme                                16.0     32.0   
group_age             18-30 ans                            20.0     32.0   
                      31-40 ans                            20.0     32.0   
                      41-50 ans                            17.0     30.0   
                      +50 ans                              16.0     28.0   
Arrêt_maladie         Aucune fois                          17.0     32.0   
                      Une fois                             16.0     29.0   
                      Deux fois ou plus                    24.0     29.0   
Horaires_tâches       Le temps prescrit                    16.0     32.0   
                      Plus                                 17.0     32.0   
                      Moins                                24.0     24.0   
SituationMatrimoniale Marié                                16.0     32.0   
                      Célibataire                          20.0     32.0   
Arrêt_travail         Jamais                               21.0     32.0   
                      Rarement                             17.0     29.0   
                      À certain moments                    18.0     32.0   
                      N'importe quand                      16.0     28.0   
Titre_Post            D.E.S.                               20.0     27.0   
                      Infirmiers                           16.0     28.0   
                      Stagiaires                           24.0     29.0   
                      Médecin spécialiste                  24.0     24.0   
                      Médecin généraliste                  22.0     27.0   
                      Aides de bloc opératoire             25.0     30.0   
                      Sage-femme                           17.0     32.0   
                      Professeur Med                       24.0     27.0   
                      Internes ou Thésards                 25.0     27.0   
                      Chargé de nettoyage                  30.0     32.0   
                      Anesthésiste réanimateur             20.0     26.0   
Hygiène_vie           Pratique d'une activité sportive     17.0     32.0   
                      Consommation d'alcool                20.0     24.0   
                      Consommation de somnifères           16.0     24.0   
                      Consommation de tabac                24.0     24.0   
Statut_salarié        Bénévole                             20.0     32.0   
                      Salarié                              16.0     32.0   
Typecontrat           Aucun contrat                        22.0     32.0   
                      Stage                                20.0     30.0   
                      CDI                                  16.0     32.0   

                                                             Mean  Median  \
Caractéristiques                                                            
sexe                  Homme                             25.583333    25.0   
                      Femme                             25.926829    26.0   
group_age             18-30 ans                         26.178571    26.0   
                      31-40 ans                         27.500000    28.5   
                      41-50 ans                         25.071429    24.0   
                      +50 ans                           21.857143    22.0   
Arrêt_maladie         Aucune fois                       26.264151    26.0   
                      Une fois                          23.200000    23.0   
                      Deux fois ou plus                 26.500000    26.5   
Horaires_tâches       Le temps prescrit                 26.370370    26.0   
                      Plus                              25.514286    26.0   
              



TABLEAU DES DECILES DE Score_soutien_social PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Percentile 1%  \
Caractéristiques                                                        
sexe                  Homme                                     20.00   
                      Femme                                     16.40   
group_age             18-30 ans                                 20.27   
                      31-40 ans                                 20.30   
                      41-50 ans                                 17.78   
                      +50 ans                                   16.12   
Arrêt_maladie         Aucune fois                               18.56   
                      Une fois                                  16.18   
                      Deux fois ou plus                         24.05   
Horaires_tâches       Le temps prescrit                         18.08   
                      Plus                                      17.34   
                      Moins                                     24.00   
SituationMatrimoniale Marié                                     16.46   
                      Célibataire                               20.17   
Arrêt_travail         Jamais                                    21.16   
                      Rarement                                  17.30   
                      À certain moments                         18.58   
                      N'importe quand                           16.48   
Titre_Post            D.E.S.                                    20.00   
                      Infirmiers                                16.80   
                      Stagiaires                                24.00   
                      Médecin spécialiste                       24.00   
                      Médecin généraliste                       22.06   
                      Aides de bloc opératoire                  25.03   
                      Sage-femme                                17.16   
                      Professeur Med                            24.00   
                      Internes ou Thésards                      25.02   
                      Chargé de nettoyage                       30.00   
                      Anesthésiste réanimateur                  20.06   
Hygiène_vie           Pratique d'une activité sportive          17.59   
                      Consommation d'alcool                     20.04   
                      Consommation de somnifères                16.08   
                      Consommation de tabac                     24.00   
Statut_salarié        Bénévole                                  20.33   
                      Salarié                                   16.30   
Typecontrat           Aucun contrat                             22.38   
                      Stage                                     20.16   
                      CDI                                       16.27   

                                                        Percentile 25%  \
Caractéristiques                                                         
sexe                  Homme                                      24.00   
                      Femme                                      24.00   
group_age             18-30 ans                                  24.00   
                      31-40 ans                                  25.50   
                      41-50 ans                                  24.00   
                      +50 ans                                    19.00   
Arrêt_maladie         Aucune fois                                24.00   
                      Une fois                                   20.00   
                      Deux fois ou plus                          25.25   
Horaires_tâches       Le temps prescrit                          24.50   
                      Plus                                       23.50   
                      Moins                                      24.00   
SituationMatrimoniale Marié                                      24.00   
                      Cél

### Tableaux statistiques Etats Psychologiques et Caractéristiques socio 

#### Fonction pour calculer un tableau de répartition simple (%)

In [ ]:

def category_distribution(data, var):
    # Validation des données
    if not isinstance(data, pd.DataFrame):
        raise TypeError('data is not a pandas.DataFrame')

    if var not in data.columns or data[var].dtype != 'category':
        raise TypeError(f'{var} not in data or not a <dtype.category>')
    
    # Calcul des fréquences et des pourcentages
    counts = data[var].value_counts()
    percentages = data[var].value_counts(normalize=True) * 100

    # Création du tableau
    distribution_table = pd.DataFrame({
        'Nombre d\'observations': counts,
        'Pourcentage (%)': percentages.round(2)
    })
        # Calcul des totaux
    total_counts = counts.sum()
    total_percentage = percentages.sum()

    # Ajout de la ligne des totaux
    distribution_table.loc['Total'] = [total_counts, total_percentage]
    
    print("REPARTITION GENERALE DES ETATS DE KARASEK DANS L'ECHANTILLON TOTAL")
    
    return distribution_table


#### Fonction pour calculer un tableau de répartition croisé avec d'autres variables (%)

In [ ]:

def cross_table(data, target_var, features_list):
    
    # Initialisation d'une liste
    tables=[]
    
    #Validation des données
    if not isinstance(data,pd.DataFrame):
        raise TypeError('data is not a pandas.DataFrame')

    if not ((target_var in data.columns) and data[target_var].dtype=='category'):
        raise ValueError(f'{target_var} not in data or not a <dtype.category>')
    
    for var in features_list:
        if not (var in data.columns and data[var].dtype in ['category', 'bool']):
            raise TypeError(f'{var} not in data.columns or not in dtype.["category","bool"]')

    #Calcul des tables croisées pour différentes variables
    for var in data.loc[:,features_list].columns:
        cross= pd.crosstab(index=data[var], columns=data[target_var], normalize='index') * 100
        cross=cross.round(2)
        cross['Total %'] = np.round(cross.sum(axis=1),1)
        cross["Nombre d'observations"]=data[var].value_counts()
        tables.append(cross)

    Summary_table = pd.concat([cross_tab for cross_tab in tables], keys=features_list)

    print("REPARTITION DES 4 ETATS PSYCHOLOGIQUES PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES")
    return Summary_table

In [ ]:
category_distribution(data_imputed,'Etat_Karasek')

REPARTITION GENERALE DES ETATS DE KARASEK DANS L'ECHANTILLON TOTAL


,Nombre d'observations,Pourcentage (%)
Etat_Karasek,,
Etat_Passif,22.0,33.85
Etat_Détendu,16.0,24.62
Etat_Stress,16.0,24.62
Etat_Actif,11.0,16.92
Total,65.0,100.00


In [ ]:
cross_table(data_imputed,'Etat_Karasek',cat_list) 

REPARTITION DES 4 ETATS PSYCHOLOGIQUES PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Etat_Karasek                                            Etat_Actif  \
sexe                  Homme                                  16.67   
                      Femme                                  17.07   
group_age             18-30 ans                              10.71   
                      31-40 ans                              25.00   
                      41-50 ans                              21.43   
                      +50 ans                                14.29   
Arrêt_maladie         Aucune fois                            20.75   
                      Une fois                                0.00   
                      Deux fois ou plus                       0.00   
Horaires_tâches       Le temps prescrit                       7.41   
                      Plus                                   22.86   
                      Moins                                  33.33   
SituationMatrimoniale Marié                                  19.15   
                      Célibataire                            11.11   
Arrêt_travail         Jamais                                 11.76   
                      Rarement                               36.36   
                      À certain moments                      16.67   
                      N'importe quand                         0.00   
Titre_Post            D.E.S.                                 20.00   
                      Infirmiers                              9.09   
                      Stagiaires                             11.11   
                      Médecin spécialiste                     0.00   
                      Médecin généraliste                    25.00   
                      Aides de bloc opératoire               75.00   
                      Sage-femme                             17.65   
                      Professeur Med                          0.00   
                      Internes ou Thésards                    0.00   
                      Chargé de nettoyage                     0.00   
                      Anesthésiste réanimateur               25.00   
Hygiène_vie           Pratique d'une activité sportive       18.33   
                      Consommation d'alcool                   0.00   
                      Consommation de somnifères              0.00   
                      Consommation de tabac                   0.00   
Statut_salarié        Bénévole                                8.82   
                      Salarié                                25.81   
Typecontrat           Aucun contrat                          10.00   
                      Stage                                  11.76   
                      CDI                                    25.00   
PsantePhysique        False                                  16.67   
                      True                                   18.18   
PsantePsychologique   False                                  18.33   
                      True                                    0.00   
Conflit_Hiérarchie    False                                  12.50   
                      True                                   44.44   
Maladies_Profil       False                                  16.92   
Conflit_collègues     False                                  14.04   
                      True                                   37.50   
Pas_de_problème_Santé False                                  15.38   
                      True                                   17.31   
Conflit_subordonnées  False                                  13.79   
                      True                                   42.86   
Pas_de_conflit        False                                  33.33   
                      True                                   12.00   

Etat_Karasek                                            Etat_Détendu  \
sexe                  Homme                                    16.67   
                      Femme                                    29.27   
gro

### Tableaux statistiques sur le Rapport Equilibre_Efforts_Récompenses de Siegrist

In [ ]:
#liste de mes variables concernées
Siegrist_list=['Score_Efforts', 'Score_Récompenses', 'Equilibre_Efforts_Récompenses']

print(f"\n")
print("TABLEAU STATISTIQUE GENERAL SUR LES SCORES DE SIEGRIST DANS L'ECHANTILLON TOTAL")
display(statistical_summary(data_imputed, Siegrist_list))

print(f"\n")
Cross_summary(data_imputed,Siegrist_list,cat_list[0:10])



TABLEAU STATISTIQUE GENERAL SUR LES SCORES DE SIEGRIST DANS L'ECHANTILLON TOTAL


,Min,Max,Mean,Median,Variance,Std Dev,Total Observations
Score_Efforts,8.000000,20.0,15.492308,16.000000,6.660096,2.580716,65.0
Score_Récompenses,9.000000,19.0,13.523077,13.000000,3.753365,1.937360,65.0
Equilibre_Efforts_Récompenses,0.727273,1.9,1.167777,1.153846,0.069490,0.263610,65.0




TABLEAU STATISTIQUE DE Score_Efforts PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Minimum  Maximum  \
Caractéristiques                                                           
sexe                  Homme                                12.0     19.0   
                      Femme                                 8.0     20.0   
group_age             18-30 ans                             8.0     19.0   
                      31-40 ans                            12.0     20.0   
                      41-50 ans                            11.0     20.0   
                      +50 ans                              12.0     19.0   
Arrêt_maladie         Aucune fois                           8.0     20.0   
                      Une fois                             10.0     19.0   
                      Deux fois ou plus                    14.0     15.0   
Horaires_tâches       Le temps prescrit                    10.0     17.0   
                      Plus                                 11.0     20.0   
                      Moins                                 8.0     17.0   
SituationMatrimoniale Marié                                10.0     20.0   
                      Célibataire                           8.0     19.0   
Arrêt_travail         Jamais                               12.0     20.0   
                      Rarement                             11.0     20.0   
                      À certain moments                     8.0     19.0   
                      N'importe quand                      10.0     16.0   
Titre_Post            D.E.S.                               17.0     19.0   
                      Infirmiers                           10.0     17.0   
                      Stagiaires                            8.0     15.0   
                      Médecin spécialiste                  15.0     15.0   
                      Médecin généraliste                  13.0     20.0   
                      Aides de bloc opératoire             16.0     18.0   
                      Sage-femme                           11.0     20.0   
                      Professeur Med                       12.0     17.0   
                      Internes ou Thésards                 15.0     15.0   
                      Chargé de nettoyage                  18.0     19.0   
                      Anesthésiste réanimateur             14.0     19.0   
Hygiène_vie           Pratique d'une activité sportive      8.0     20.0   
                      Consommation d'alcool                15.0     17.0   
                      Consommation de somnifères           14.0     15.0   
                      Consommation de tabac                14.0     14.0   
Statut_salarié        Bénévole                              8.0     20.0   
                      Salarié                              10.0     20.0   
Typecontrat           Aucun contrat                         8.0     19.0   
                      Stage                                13.0     20.0   
                      CDI                                  10.0     20.0   

                                                             Mean  Median  \
Caractéristiques                                                            
sexe                  Homme                             16.791667    17.0   
                      Femme                             14.731707    14.0   
group_age             18-30 ans                         14.821429    15.0   
                      31-40 ans                         16.687500    17.0   
                      41-50 ans                         15.357143    15.5   
                      +50 ans                           15.714286    16.0   
Arrêt_maladie         Aucune fois                       15.566038    16.0   
                      Une fois                          15.300000    16.0   
                      Deux fois ou plus                 14.500000    14.5   
Horaires_tâches       Le temps prescrit                 14.259259    14.0   
                      Plus                              16.685714    17.0   
              



TABLEAU DES DECILES DE Score_Efforts PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


...

Percentile 1%  \
Caractéristiques                                                        
sexe                  Homme                                     10.46   
                      Femme                                      9.00   
group_age             18-30 ans                                 11.27   
                      31-40 ans                                 11.15   
                      41-50 ans                                  9.13   
                      +50 ans                                    9.06   
Arrêt_maladie         Aucune fois                                9.52   
                      Une fois                                   9.09   
                      Deux fois ou plus                         13.02   
Horaires_tâches       Le temps prescrit                          9.78   
                      Plus                                       9.34   
                      Moins                                     11.04   
SituationMatrimoniale Marié                                      9.00   
                      Célibataire                               11.17   
Arrêt_travail         Jamais                                     9.32   
                      Rarement                                  10.20   
                      À certain moments                         10.29   
                      N'importe quand                            9.18   
Titre_Post            D.E.S.                                    12.00   
                      Infirmiers                                 9.00   
                      Stagiaires                                11.08   
                      Médecin spécialiste                       17.00   
                      Médecin généraliste                       11.06   
                      Aides de bloc opératoire                  14.00   
                      Sage-femme                                10.32   
                      Professeur Med                            12.02   
                      Internes ou Thésards                      13.01   
                      Chargé de nettoyage                       13.04   
                      Anesthésiste réanimateur                  10.06   
Hygiène_vie           Pratique d'une activité sportive           9.59   
                      Consommation d'alcool                     12.01   
                      Consommation de somnifères                 9.08   
                      Consommation de tabac                     14.00   
Statut_salarié        Bénévole                                   9.66   
                      Salarié                                    9.30   
Typecontrat           Aucun contrat                              9.38   
                      Stage                                     12.00   
                      CDI                                        9.27   

                                                        Percentile 25%  \
Caractéristiques                                                         
sexe                  Homme                                      12.75   
                      Femme                                      13.00   
group_age             18-30 ans                                  12.75   
                      31-40 ans                                  13.00   
                      41-50 ans                                  12.25   
                      +50 ans                                    11.00   
Arrêt_maladie         Aucune fois                                13.00   
                      Une fois                                   11.25   
                      Deux fois ou plus                          13.50   
Horaires_tâches       Le temps prescrit                          13.00   
                      Plus                                       12.00   
                      Moins                                      12.00   
SituationMatrimoniale Marié                                      12.50   
                      Cél

TABLEAU STATISTIQUE DE Equilibre_Efforts_Récompenses PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Minimum   Maximum  \
Caractéristiques                                                             
sexe                  Homme                             0.882353  1.900000   
                      Femme                             0.727273  1.900000   
group_age             18-30 ans                         0.727273  1.461538   
                      31-40 ans                         0.842105  1.583333   
                      41-50 ans                         0.733333  1.900000   
                      +50 ans                           0.923077  1.900000   
Arrêt_maladie         Aucune fois                       0.727273  1.900000   
                      Une fois                          0.769231  1.900000   
                      Deux fois ou plus                 0.933333  1.153846   
Horaires_tâches       Le temps prescrit                 0.769231  1.555556   
                      Plus                              0.733333  1.900000   
                      Moins                             0.727273  1.307692   
SituationMatrimoniale Marié                             0.733333  1.900000   
                      Célibataire                       0.727273  1.461538   
Arrêt_travail         Jamais                            0.842105  1.777778   
                      Rarement                          0.733333  1.900000   
                      À certain moments                 0.727273  1.900000   
                      N'importe quand                   0.769231  1.555556   
Titre_Post            D.E.S.                            1.384615  1.583333   
                      Infirmiers                        0.769231  1.777778   
                      Stagiaires                        0.727273  1.250000   
                      Médecin spécialiste               0.882353  0.882353   
                      Médecin généraliste               0.944444  1.545455   
                      Aides de bloc opératoire          1.142857  1.285714   
                      Sage-femme                        0.733333  1.900000   
                      Professeur Med                    1.000000  1.307692   
                      Internes ou Thésards              1.071429  1.153846   
                      Chargé de nettoyage               1.187500  1.461538   
                      Anesthésiste réanimateur          1.000000  1.900000   
Hygiène_vie           Pratique d'une activité sportive  0.727273  1.900000   
                      Consommation d'alcool             1.153846  1.416667   
                      Consommation de somnifères        0.882353  1.555556   
                      Consommation de tabac             1.000000  1.000000   
Statut_salarié        Bénévole                          0.727273  1.777778   
                      Salarié                           0.733333  1.900000   
Typecontrat           Aucun contrat                     0.727273  1.777778   
                      Stage                             0.875000  1.416667   
                      CDI                               0.733333  1.900000   

                                                            Mean    Median  \
Caractéristiques                                                             
sexe                  Homme                             1.266660  1.222527   
                      Femme                             1.109893  1.000000   
group_age             18-30 ans                         1.111312  1.076923   
                      31-40 ans                         1.172525  1.207143   
                      41-50 ans                         1.188675  1.100000   
                      +50 ans                           1.340982  1.250000   
Arrêt_maladie         Aucune fois                       1.155588  1.166667   
                      Une fois                          1.257212  1.228022   
                      Deux fois ou plus                 1.043590  1.043590   
Horaires_tâches       Le temps prescrit                 1.064738  1.000000   




TABLEAU DES DECILES DE Equilibre_Efforts_Récompenses PAR CARACTERISTIQUES SOCIO-DEMOGRAPHIQUES


Percentile 1%  \
Caractéristiques                                                        
sexe                  Homme                                  0.909412   
                      Femme                                  0.729697   
group_age             18-30 ans                              0.738601   
                      31-40 ans                              0.849123   
                      41-50 ans                              0.750667   
                      +50 ans                                0.936264   
Arrêt_maladie         Aucune fois                            0.730424   
                      Une fois                               0.778000   
                      Deux fois ou plus                      0.935538   
Horaires_tâches       Le temps prescrit                      0.788178   
                      Plus                                   0.778667   
                      Moins                                  0.732727   
SituationMatrimoniale Marié                                  0.749846   
                      Célibataire                            0.752386   
Arrêt_travail         Jamais                                 0.849591   
                      Rarement                               0.753333   
                      À certain moments                      0.767697   
                      N'importe quand                        0.776018   
Titre_Post            D.E.S.                                 1.384615   
                      Infirmiers                             0.785165   
                      Stagiaires                             0.739091   
                      Médecin spécialiste                    0.882353   
                      Médecin généraliste                    0.946111   
                      Aides de bloc opératoire               1.144571   
                      Sage-femme                             0.750737   
                      Professeur Med                         1.004615   
                      Internes ou Thésards                   1.072253   
                      Chargé de nettoyage                    1.187500   
                      Anesthésiste réanimateur               1.007500   
Hygiène_vie           Pratique d'une activité sportive       0.730848   
                      Consommation d'alcool                  1.156474   
                      Consommation de somnifères             0.889085   
                      Consommation de tabac                  1.000000   
Statut_salarié        Bénévole                               0.776023   
                      Salarié                                0.744103   
Typecontrat           Aucun contrat                          0.763258   
                      Stage                                  0.884333   
                      CDI                                    0.743026   

                                                        Percentile 25%  \
Caractéristiques                                                         
sexe                  Homme                                   1.153846   
                      Femme                                   0.928571   
group_age             18-30 ans                               0.983333   
                      31-40 ans                               0.986111   
                      41-50 ans                               0.929762   
                      +50 ans                                 1.186813   
Arrêt_maladie         Aucune fois                             0.944444   
                      Une fois                                1.015625   
                      Deux fois ou plus                       0.988462   
Horaires_tâches       Le temps prescrit                       0.925824   
                      Plus                                    1.102381   
                      Moins                                   0.863636   
SituationMatrimoniale Marié                                   0.938889   
                      Cél